<h1>0. Clean the environment and install necessary packages</h1>

In [51]:
#Clean the environment
import sys
sys.modules[__name__].__dict__.clear()

In [1]:
#Import all the necessary packages
import types
import pandas as pd
import numpy as np
import geopy.distance



import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

def __iter__(self): return 0
from botocore.client import Config
import ibm_boto3


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

<h1>1. Data processing</h1>

<h3>1.1 Loading the data</h1>

Firstly, we load the data including neighborhoods in NYC and their coordinates.

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
NYC_coordinates = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    NYC_coordinates = NYC_coordinates.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    NYC_coordinates.shape

Data downloaded!


Secondly, we load the data on NYC community Districts.

In [3]:
# The code was removed by Watson Studio for sharing.

Let's give the dataset an informative title and preview it.

In [4]:
NYC_data_raw=df_data_10

print(NYC_data_raw.shape)
NYC_data_raw.head()

(59, 196)


,the_geom,female_50_54,the_geom_webmercator,shared_puma,fp_500_mhhi,fp_100_mhhi,cd_tot_resunits,fp_500_resunits,fp_500_bldg,fp_100_resunits,...,acs_tooltip_2,acs_tooltip_3,fp_100_ownerocc_value,fp_500_ownerocc_value,fp_100_mortg_value,fp_500_mortg_value,fp_100_cost_burden_value,fp_500_cost_burden_value,fp_100_rent_burden_value,fp_500_rent_burden_value
0,NaN,2.2,NaN,False,82000.0,93000.0,82471,22491,3495,10046,...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,2618,4495,1812,2880,918,1680,3789,8090
1,NaN,3.4,NaN,False,NaN,NaN,45824,0,0,0,...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,0,0,0,0,0,0,0,0
2,NaN,4.4,NaN,False,35000.0,40000.0,60790,5053,3906,2583,...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,2783,4094,2174,3189,1345,2068,957,1573
3,NaN,3.2,NaN,False,47000.0,47000.0,44616,1035,102,989,...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,684,1015,305,368,91,116,796,1089
4,NaN,3.0,NaN,False,70000.0,NaN,67704,194,87,0,...,American Community Survey (ACS) 2013-2017 5-ye...,2010 Census population counts for floodplain a...,0,424,0,312,0,192,0,1197


The dataset cosists of 59 community districts (rows) and 196 variables (columns). 
<br> We analyze only variables that might be the most relevant when choosing a place to live, such as communiting time, crime, availability of parks etc.
<br> Moreover, we want to obtain the data for the neighborhoods. Each community district has its neighborhoods listed in variable 'neighborhoods'.
<br> We would need to split these names into separate rows and then assign to them the data of the corresponding community district. I.e we need to 'explode' the column into multiple rows. To do so, we need two datasets - first with the selected indicators and name of CD, the second one with the names of the neighborhoods included in the CD and name of the CD (we will need it to merge these two datasets).

In [5]:
NYC_Neighborhoods=NYC_data_raw[['cd_full_title', 'neighborhoods']]

NYC_Indicators=NYC_data_raw[['cd_full_title',
                               'pct_lot_area_open_space',
                               'crime_per_1000',
                               'pct_clean_strts',
                               'pct_hh_rent_burd',
                               'mean_commute',
                               'unemployment_cd',
                               'poverty_rate',
                               'pct_lot_area_industrial_manufacturing']]
NYC_Indicators.head()

,cd_full_title,pct_lot_area_open_space,crime_per_1000,pct_clean_strts,pct_hh_rent_burd,mean_commute,unemployment_cd,poverty_rate,pct_lot_area_industrial_manufacturing
0,Brooklyn Community District 1,0.050759,16.143842,92.5,40.6,34,4.3,17.3,0.248568
1,Queens Community District 9,0.021635,7.364655,95.0,48.3,45,5.2,21.9,0.030127
2,Queens Community District 13,0.132616,8.711107,97.1,43.8,47,5.4,13.2,0.021895
3,Bronx Community District 8,0.162831,8.827964,97.1,43.8,45,5.9,15.2,0.005224
4,Brooklyn Community District 3,0.033102,20.585544,90.7,46.1,41,8.3,22.6,0.032322


<h2>1.2 Splitting a column with neighborhoods into rows</h2>

To explode the'neighborhoods' column, we split the string variable and create a new dataframe from the series with 'neighborhood' as the index. Then, we get rid of the secondary index. To do this, we will make 'cd_full_title' as a column (it can't be an index since the values will be duplicate). Lastly, we name columns as in the original dataset.

In [6]:
NYC_Neighborhoods= pd.DataFrame(NYC_Neighborhoods.neighborhoods.str.split(',').tolist(), index=NYC_Neighborhoods.cd_full_title).stack()
NYC_Neighborhoods = NYC_Neighborhoods.reset_index([0, 'cd_full_title'])
NYC_Neighborhoods.columns = ['cd_full_title', 'neighborhoods']

We merge the dataset containing selected indicators and splitted neighborhoods based on the community district name. As a result we obtain a dataset with 380 rows corresponding to the neighborhoods. Each neighborhood has assigned idicator values corresponding to the indicator values of CD where they belong.

In [7]:
NYC_Neighborhoods_data=NYC_Neighborhoods.merge(NYC_Indicators,on='cd_full_title')
NYC_Neighborhoods_data.shape

(380, 10)

Now we can rename variables to give them shorter and maybe more informative titles.

In [10]:
NYC_Neighborhoods_data.rename(columns={'cd_full_title': 'District',
                                      'pct_lot_area_open_space': 'Open_area',
                                       'crime_per_1000': 'Crime',
                                       'pct_hh_rent_burd': 'Rent_Burden',
                                       'mean_commute': 'Commute_time',
                                       'unemployment_cd':'Unemployment',
                                       'neighborhoods':'Neighborhood',
                                        'pct_clean_strts': 'Street_clean',
                                        'poverty_rate': 'Poverty',
                                         'pct_lot_area_industrial_manufacturing': 'Industrial_area',
                                          }, inplace=True)



In [11]:
NYC_Neighborhoods_data.head()

,District,Neighborhood,Open_area,Crime,Street_clean,Rent_Burden,Commute_time,Unemployment,Poverty,Industrial_area
0,Brooklyn Community District 1,East Williamsburg,0.050759,16.143842,92.5,40.6,34,4.3,17.3,0.248568
1,Brooklyn Community District 1,Greenpoint,0.050759,16.143842,92.5,40.6,34,4.3,17.3,0.248568
2,Brooklyn Community District 1,Northside,0.050759,16.143842,92.5,40.6,34,4.3,17.3,0.248568
3,Brooklyn Community District 1,Southside,0.050759,16.143842,92.5,40.6,34,4.3,17.3,0.248568
4,Brooklyn Community District 1,Williamsburg,0.050759,16.143842,92.5,40.6,34,4.3,17.3,0.248568


<h2>1.3 Merging the datasets with coordinates and indicators</h2>

To ensure that the neighborhood names in the dataset with coordinates and the dataset with the indicators are matched properly, we strip the names of spaces.

In [12]:
NYC_Neighborhoods_data['Neighborhood'] = NYC_Neighborhoods_data['Neighborhood'].str.strip()
NYC_coordinates['Neighborhood'] = NYC_coordinates['Neighborhood'].str.strip()

Now, we can merge the datasets and check the dimensions.

In [13]:
NYC_full_data=pd.merge(NYC_Neighborhoods_data,NYC_coordinates,on='Neighborhood')
NYC_full_data.shape

(315, 13)

It seems that some of the names were duplicated and as a result we got more than 306 neighborhoods. Let's check the duplicated values.

In [14]:
NYC_full_data[NYC_full_data.duplicated(['Neighborhood'])]

,District,Neighborhood,Open_area,Crime,Street_clean,Rent_Burden,Commute_time,Unemployment,Poverty,Industrial_area,Borough,Latitude,Longitude
5,Queens Community District 10,Ozone Park,0.143426,9.705540,97.2,50.8,45,6.5,18.9,0.005698,Queens,40.680708,-73.843203
25,Brooklyn Community District 16,Broadway Junction,0.053767,12.588770,92.2,51.2,47,7.6,28.2,0.047743,Brooklyn,40.677861,-73.903317
45,Bronx Community District 5,Fordham,0.087306,13.975545,92.1,59.8,42,8.1,34.4,0.015496,Bronx,40.860997,-73.896427
49,Bronx Community District 5,University Heights,0.087306,13.975545,92.1,59.8,42,8.1,34.4,0.015496,Bronx,40.855727,-73.910416
61,Brooklyn Community District 13,Gravesend,0.383340,8.662746,93.8,45.5,46,6.0,24.0,0.009453,Brooklyn,40.595260,-73.973471
62,Brooklyn Community District 15,Gravesend,0.026119,7.905350,97.5,47.1,44,4.9,19.4,0.003829,Brooklyn,40.595260,-73.973471
66,Brooklyn Community District 14,Ocean Parkway,0.036882,8.833473,93.4,50.1,42,5.5,22.1,0.003529,Brooklyn,40.613060,-73.968367
70,Brooklyn Community District 15,Homecrest,0.026119,7.905350,97.5,47.1,44,4.9,19.4,0.003829,Brooklyn,40.598525,-73.959185
73,Staten Island Community District 1,West Brighton,0.151974,9.734195,96.1,48.6,43,3.9,21.2,0.032137,Staten Island,40.631879,-74.107182
76,Brooklyn Community District 17,Flatbush,0.057141,14.626822,95.8,49.3,46,5.7,19.0,0.036117,Brooklyn,40.636326,-73.958401


There are too many rows including Chelsea, Bay Terrace, Sunnyside Gravesend and Murray Hill neigborhoods. We delete those that got assigned wrong borough.

In [15]:
NYC_full_data= NYC_full_data.drop(NYC_full_data.index[[61,84,86,121,122,133,135,136,142,144,145,146]])

<h1> 2. Explanatory data analysis</h1>

We start primary analysis with visualising the map of neighborhoods. Firstly, we get the coordinates of New York using geocode.

In [16]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


Then, we map the neighborhoods using folium package.

In [17]:

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(NYC_full_data['Latitude'], 
                                           NYC_full_data['Longitude'],
                                           NYC_full_data['Borough'], 
                                           NYC_full_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    if neighborhood=='Bay Terrace':
        folium.CircleMarker(
                [lat, lng],
                radius=6,
                popup=label,
                color='red',
                fill=True,
                fill_color='#FF0000',
                fill_opacity=0.7,
                parse_html=False).add_to(map_newyork)
    else: folium.CircleMarker(
                [lat, lng],
                radius=3,
                popup=label,
                color='darkblue',
                fill=True,
                fill_color='#3390FF',
                fill_opacity=0.7,
                parse_html=False).add_to(map_newyork)
    
    folium.Circle([40.741895, -73.989308],
                    radius=13000,
                    color='red',
                    fill=True,
                    fill_color='#FF0000',
                    fill_opacity=0.001).add_to(map_newyork)

In [18]:
map_newyork

Let's explore the characteristics of each Borough by grouping the data using mean and compare them to the indicator values of the current neighborhood of our client.

In [19]:
NYC_full_data.groupby('Borough').mean().reset_index()

,Borough,Open_area,Crime,Street_clean,Rent_Burden,Commute_time,Unemployment,Poverty,Industrial_area,Latitude,Longitude
0,Bronx,0.124235,13.605862,95.301887,48.764151,43.830189,7.386792,24.007547,0.040005,40.850820,-73.873735
1,Brooklyn,0.136358,11.366159,93.746377,44.779710,42.710145,5.465217,20.420290,0.051886,40.646101,-73.952079
2,Manhattan,0.122914,19.313509,96.156757,36.575676,30.459459,3.975676,12.951351,0.012707,40.762799,-73.975752
3,Queens,0.167045,9.162469,96.580247,46.992593,43.864198,5.024691,19.344444,0.044659,40.707201,-73.826134
4,Staten Island,0.213780,6.764254,98.239683,45.911111,42.809524,3.676190,16.384127,0.027841,40.589165,-74.136091


In [21]:
import seaborn as sns
sns.set(style="ticks")

df = NYC_full_data[['Open_area',
                    'Crime',
                    'Street_clean',
                    'Rent_Burden',
                    'Commute_time',
                    'Unemployment',
                    'Poverty',
                    'Industrial_area',
                    'Borough']]

sns.pairplot(df,hue='Borough')

**Based on the table we see that the crime rates are highest in Manhattan Borough. Manhattan is also characterized by lower commuting time, lower poverty and smaller open areas. All these characteristics are pretty common for the centers of the big cities. The lower value of rent burden can be explained by wealthy inhabitants - although rent prices are high, they are still affordable for the rich. 
The Borough of Staten Island, were the Bay Terrace is located, features lots of open areas, low crime and unemployment. It seems that neighborhoods of Queens district share similar characteristics. The next possible candidates could be located in Brookly Borough.**

<h2>2.2 Cluster analysis<h2>

We import necessary packages and subset the data that include numeric values and can be passed on to cluster analysis.

In [22]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn import preprocessing

In [23]:
NYC_cluster_data=NYC_full_data[['Open_area',
                                'Crime',
                                'Street_clean',
                                'Rent_Burden',
                                'Commute_time',
                                'Unemployment',
                                'Poverty',
                                'Industrial_area']]

Since the ranges of the indicators vary, we normalize the data before clustering.

In [24]:
NYC_cluster_data = preprocessing.normalize(NYC_cluster_data)

In [25]:
kclusters = 4
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NYC_cluster_data)
kmeans.labels_


array([2, 2, 2, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 2, 3, 2, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 3, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2,
       0, 0, 0, 3, 3, 3, 3, 2, 2, 2, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3,

We append the assigned clusters to the full dataset.

In [26]:
NYC_full_data['Cluster']=kmeans.labels_
NYC_full_data.head()

,District,Neighborhood,Open_area,Crime,Street_clean,Rent_Burden,Commute_time,Unemployment,Poverty,Industrial_area,Borough,Latitude,Longitude,Cluster
0,Brooklyn Community District 1,East Williamsburg,0.050759,16.143842,92.5,40.6,34,4.3,17.3,0.248568,Brooklyn,40.708492,-73.938858,2
1,Brooklyn Community District 1,Greenpoint,0.050759,16.143842,92.5,40.6,34,4.3,17.3,0.248568,Brooklyn,40.730201,-73.954241,2
2,Brooklyn Community District 1,Williamsburg,0.050759,16.143842,92.5,40.6,34,4.3,17.3,0.248568,Brooklyn,40.707144,-73.958115,2
3,Queens Community District 9,Kew Gardens,0.021635,7.364655,95.0,48.3,45,5.2,21.9,0.030127,Queens,40.705179,-73.829819,0
4,Queens Community District 9,Ozone Park,0.021635,7.364655,95.0,48.3,45,5.2,21.9,0.030127,Queens,40.680708,-73.843203,0


In [27]:
pd.DataFrame(NYC_full_data['Cluster'].value_counts())

,Cluster
0,153
3,108
2,39
1,3


Now, we map the neighborhoods with colors corresponding to their cluster groups to get an insight of the geographical spread of the clusters.

In [28]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
import matplotlib.cm as cm
import matplotlib.colors as colors
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NYC_full_data['Latitude'],
                                  NYC_full_data['Longitude'],
                                  NYC_full_data['Neighborhood'],
                                  NYC_full_data['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's now see what are the mean indicator values of the different clusters.

In [29]:
NYC_full_data.groupby(['Cluster']).mean()

,Open_area,Crime,Street_clean,Rent_Burden,Commute_time,Unemployment,Poverty,Industrial_area,Latitude,Longitude
Cluster,,,,,,,,,,
0,0.131053,11.427732,94.700654,49.691503,43.098039,5.883660,24.069281,0.044395,40.724982,-73.925946
1,0.031681,87.226379,97.600000,34.200000,26.000000,4.100000,10.800000,0.026816,40.747625,-73.987110
2,0.068131,13.969668,95.846154,33.489744,31.000000,3.897436,10.889744,0.056090,40.728685,-73.981209
3,0.229041,7.716151,97.864815,43.656481,44.129630,4.535185,15.000000,0.022977,40.654767,-73.960154


Remove unnecessary column

In [35]:
NYC_full_data.drop(['District'],axis=1,inplace=True)

In [31]:
NYC_full_data[NYC_full_data['Neighborhood']=='Bay Terrace']

,Neighborhood,Open_area,Crime,Street_clean,Rent_Burden,Commute_time,Unemployment,Poverty,Industrial_area,Borough,Latitude,Longitude,Cluster
134,Bay Terrace,0.112828,7.653031,98.2,55.4,41,3.5,25.2,0.041905,Staten Island,40.553988,-74.139166,0


One of the main requirements of our client was the commuting time - it should be no higher than in their current neighborhood, Bay Terrace. We filter the data to get the neighborhoods from the same cluster as Bay Terrace with desired commuting time.

In [34]:
Chosen_neighborhoods=NYC_full_data[(NYC_full_data.Cluster==0)&(NYC_full_data.Commute_time<=41)]
Chosen_neighborhoods

,Neighborhood,Open_area,Crime,Street_clean,Rent_Burden,Commute_time,Unemployment,Poverty,Industrial_area,Borough,Latitude,Longitude,Cluster
23,Bushwick,0.157937,11.884736,92.1,48.6,40,8.8,25.1,0.050248,Brooklyn,40.698116,-73.925258,0
50,Concourse,0.183724,16.109177,93.9,54.6,41,7.4,32.3,0.032312,Bronx,40.834284,-73.915589,0
51,Concourse Village,0.183724,16.109177,93.9,54.6,41,7.4,32.3,0.032312,Bronx,40.824780,-73.915847,0
52,Mount Eden,0.183724,16.109177,93.9,54.6,41,7.4,32.3,0.032312,Bronx,40.843826,-73.916556,0
63,Borough Park,0.056145,6.022785,86.3,60.6,37,3.5,28.3,0.024619,Brooklyn,40.633131,-73.990498,0
64,Kensington,0.056145,6.022785,86.3,60.6,37,3.5,28.3,0.024619,Brooklyn,40.642382,-73.980421,0
65,Ocean Parkway,0.056145,6.022785,86.3,60.6,37,3.5,28.3,0.024619,Brooklyn,40.613060,-73.968367,0
114,Long Island City,0.156393,7.547115,93.5,44.2,38,3.7,20.4,0.263630,Queens,40.750217,-73.939202,0
117,Blissville,0.156393,7.547115,93.5,44.2,38,3.7,20.4,0.263630,Queens,40.737251,-73.932442,0
118,Hunters Point,0.156393,7.547115,93.5,44.2,38,3.7,20.4,0.263630,Queens,40.743414,-73.953868,0


**As expected, many of the neighborhoods in the same cluster as Bay Terrace are located in Queens borough. Now, we will examine which of the 18 shortlisted neighborhoods meet the rest of the requirements from the client.**

<h2>2.1 Requirement 1- the distance from the new office located on the Wall St. 111.  needs to be lower than 15 km. </h2>

To get the distances from the selected neighboroods, we use geolocator package.

In [36]:
import geopy.distance

To use the geocode function, we create the list with the shortlisted neighborhoods.

In [37]:
Chosen_neighborhoods_list=list(Chosen_neighborhoods['Neighborhood'])

Next, we loop over each neighborhood, get its coordinates and calculate the distance from the office using distance.vincenty function.

In [45]:
#define the address of the office

work_address = '111 Wall Street, New York City, NY'
work_location = geolocator.geocode(work_address,timeout=None)
work_distance=pd.DataFrame(columns=['Neighborhood','Distance'])

#loop over each neighborhood in the list and display the distance from the work address
for neighborhood in Chosen_neighborhoods.iterrows():

    neigh_address=str(neighborhood)+", New York City, NY'"
    geolocator = Nominatim(user_agent="ny_explorer")
    neigh_location = geolocator.geocode(neigh_address)
    if neigh_location is not None and neigh_location.longitude is not None:
        distance=geopy.distance.vincenty((work_location.latitude,work_location.longitude),
                                         (neigh_location.latitude,neigh_location.longitude))
        
        print(str(neighborhood)+": "+str(distance))
        
        

           

GeocoderTimedOut: Service timed out

**Unfortunately, geopy blocks my requests because of their usage policy (since I sent a lot of requests). The code works, but it might be necessary to wait some time until running it again**

**As we see, Concourse, Concourse Village, Auburndale, Beechhurst, College Point, Malba and Whitestone neighborhoods do not meet this requirement, therefore we exclude them from the dataset with the shortlisted neighborhoods.**

In [39]:
Chosen_neighborhoods=Chosen_neighborhoods.loc[[23,63,64,65,125,126,127,134,139,141]]
Chosen_neighborhoods_list=list(Chosen_neighborhoods['Neighborhood'])

<h1>2.2 Requirement 1- Check special schools nearby</h1>

The next requirement is the immediate proximity of special school. To search for such venues nearby neighborhood, we use Foursquare API.

In [40]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

Declare the credentials, version and number of limit on number of venues.

In [41]:
CLIENT_ID = 'U2VJUAKYYBXB45MHT5CEOVTTPOVRGICKH124CTKYABJUZOQK' # your Foursquare ID
CLIENT_SECRET = '1QXRWA355WNDRGQQLNBLOI2CVBMDREHGMRAJPQWVXLBDUQB4' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 10

Now we create a loop that would get the coordinates of each of the listed neighborhoods, search for the number of special schools in the closest proximity and print out the result.**

In [45]:
for neighborhood in Chosen_neighborhoods_list:
    
    #get the address of the neigborhood
    address=str(neighborhood)+", New York City, NY'"
    
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    
    # create the query
    search_query = 'special school'
    radius = 400
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    
    venues = results['response']['venues']
    
    # tranform venues into a dataframe
    dataframe = json_normalize(venues)
    
    #prnt the number of special schools
    if dataframe.empty:
        print(neighborhood+": No venues!")
    else:
        print(neighborhood+": "+str(len(dataframe.name)))
    


Bushwick: 5
Borough Park: 3
Kensington: 1
Ocean Parkway: 2
East Elmhurst: 3
Jackson Heights: 5
North Corona: No venues!
Bay Terrace: No venues!
Flushing: 1
Murray Hill: 10


**Out of the 10 selected neighborhoods, one (except for Bay Terrace) does not have any special school in the immediate distance. We exclude it from the further analysis.**

In [359]:
Chosen_neighborhoods=Chosen_neighborhoods[Chosen_neighborhoods.Neighborhood != 'North Corona']
Chosen_neighborhoods_list=list(Chosen_neighborhoods['Neighborhood'])

<h1>2.3 Requitement 3 - Nearby venues</h1>

The client wanted the new neighborhood to have similar venues as Bay Terrace if possible. Again, we can get the data on the existing venues using Foursquare API.**

In [40]:
#Send the GET request and examine the resutls
results = requests.get(url).json()

In [41]:
# define function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [42]:
#Create a function that would send the get request, clean the json file and structure it into pandas dataframe for every shortlisted neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
#Pass on the selected neighborhoods to the created function and display the result.
chosen_neighborhood_venues = getNearbyVenues(names=Chosen_neighborhoods['Neighborhood'],
                                   latitudes=Chosen_neighborhoods['Latitude'],
                                   longitudes=Chosen_neighborhoods['Longitude']
                                      )

Bushwick
Concourse
Concourse Village
Mount Eden
Borough Park
Kensington
Ocean Parkway
Long Island City
Blissville
Hunters Point
Sunnyside
Sunnyside
Sunnyside Gardens
Woodside
East Elmhurst
Jackson Heights
North Corona
Auburndale
Bay Terrace
Beechhurst
College Point
Flushing
Malba
Murray Hill
Queensboro Hill
Whitestone
Central Harlem
East Harlem
Inwood
Washington Heights
Hamilton Heights
Manhattanville
Morningside Heights


In [405]:
chosen_neighborhood_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bushwick,40.698116,-73.925258,Mood Ring,40.697888,-73.926955,Bar
1,Bushwick,40.698116,-73.925258,Maite,40.699091,-73.926838,Latin American Restaurant
2,Bushwick,40.698116,-73.925258,The Wheelhouse,40.699997,-73.924401,Sandwich Place
3,Bushwick,40.698116,-73.925258,Taqueria Sofia,40.700242,-73.925575,Mexican Restaurant
4,Bushwick,40.698116,-73.925258,Le Garage,40.699312,-73.926591,French Restaurant


In [406]:
# one hot encoding
chosen_neighborhood_onehot = pd.get_dummies(chosen_neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chosen_neighborhood_onehot['Neighborhood'] = chosen_neighborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [chosen_neighborhood_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = chosen_neighborhood_onehot[fixed_columns]

chosen_neighborhood_onehot.head()

,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bistro,...,Train Station,Used Bookstore,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Neighborhood
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Bushwick
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Bushwick
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Bushwick
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Bushwick
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Bushwick


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.**

In [407]:
chosen_neighborhood_grouped = chosen_neighborhood_onehot.groupby('Neighborhood').mean().reset_index()
chosen_neighborhood_grouped

,Neighborhood,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,...,Toy / Game Store,Train Station,Used Bookstore,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Bay Terrace,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.090909,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
1,Borough Park,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.176471,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
2,Bushwick,0.012821,0.000000,0.012821,0.000000,0.000000,0.012821,0.038462,0.000000,0.102564,...,0.000000,0.000000,0.012821,0.012821,0.025641,0.00,0.00,0.000000,0.000000,0.000000
3,East Elmhurst,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
4,Flushing,0.000000,0.000000,0.000000,0.029851,0.029851,0.000000,0.044776,0.000000,0.000000,...,0.014925,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.014925
5,Jackson Heights,0.012195,0.012195,0.000000,0.012195,0.000000,0.000000,0.048780,0.000000,0.012195,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.012195,0.012195,0.000000
6,Kensington,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.028571,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
7,Murray Hill,0.020000,0.000000,0.000000,0.000000,0.000000,0.020000,0.020000,0.000000,0.020000,...,0.000000,0.000000,0.000000,0.000000,0.010000,0.01,0.01,0.000000,0.000000,0.000000
8,Ocean Parkway,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000


Let's print each neighborhood along with the top 10 most common venues

In [408]:
num_top_venues = 10

for hood in chosen_neighborhood_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =chosen_neighborhood_grouped[chosen_neighborhood_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bay Terrace----
                venue  freq
0         Supermarket  0.18
1          Playground  0.09
2          Donut Shop  0.09
3  Italian Restaurant  0.09
4    Insurance Office  0.09
5      Shipping Store  0.09
6       Train Station  0.09
7        Liquor Store  0.09
8    Sushi Restaurant  0.09
9  Salon / Barbershop  0.09


----Borough Park----
                 venue  freq
0          Pizza Place  0.18
1                 Bank  0.18
2             Pharmacy  0.12
3  American Restaurant  0.06
4                Hotel  0.06
5       Farmers Market  0.06
6           Restaurant  0.06
7        Deli / Bodega  0.06
8          Coffee Shop  0.06
9   Chinese Restaurant  0.06


----Bushwick----
                    venue  freq
0                     Bar  0.10
1           Deli / Bodega  0.08
2      Mexican Restaurant  0.06
3             Coffee Shop  0.06
4             Pizza Place  0.05
5          Discount Store  0.04
6                  Bakery  0.04
7  Thrift / Vintage Store  0.04
8      Italian Restaura

Let's put that into a *pandas* dataframe

In [3]:
#write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [410]:
#create the new dataframe and display the top 50 venues for each neighborhood
num_top_venues = 50

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = chosen_neighborhood_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(chosen_neighborhood_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Bay Terrace,Supermarket,Italian Restaurant,Liquor Store,Insurance Office,Playground,Salon / Barbershop,Shipping Store,Sushi Restaurant,Donut Shop,...,Health & Beauty Service,Hotel Bar,Dumpling Restaurant,Indian Restaurant,Ice Cream Shop,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Food
1,Borough Park,Bank,Pizza Place,Pharmacy,American Restaurant,Coffee Shop,Bistro,Hotel,Deli / Bodega,Restaurant,...,Farmers Market,Chinese Restaurant,Food,Gastropub,Food & Drink Shop,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store
2,Bushwick,Bar,Deli / Bodega,Mexican Restaurant,Coffee Shop,Pizza Place,Bakery,Discount Store,Thrift / Vintage Store,Diner,...,Chinese Restaurant,New American Restaurant,Sandwich Place,Italian Restaurant,Pakistani Restaurant,Cuban Restaurant,Playground,Pet Store,Creperie,Dive Bar
3,East Elmhurst,Donut Shop,Rental Car Location,Lake,Supermarket,Coffee Shop,Chinese Restaurant,Pizza Place,Gas Station,Hotel Bar,...,Fast Food Restaurant,Food,Farmers Market,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Event Space,Empanada Restaurant
4,Flushing,Bubble Tea Shop,Korean Restaurant,Hotpot Restaurant,Chinese Restaurant,Bakery,Karaoke Bar,Asian Restaurant,Gym,Construction & Landscaping,...,BBQ Joint,Hotel,Gym / Fitness Center,Gift Shop,Resort,Rental Service,Ice Cream Shop,Historic Site,Deli / Bodega,Donut Shop
5,Jackson Heights,Latin American Restaurant,South American Restaurant,Peruvian Restaurant,Mobile Phone Shop,Bakery,Thai Restaurant,Spanish Restaurant,Mexican Restaurant,Diner,...,Clothing Store,Empanada Restaurant,Pizza Place,Grocery Store,Pharmacy,Cuban Restaurant,Cosmetics Shop,Colombian Restaurant,Coffee Shop,Deli / Bodega
6,Kensington,Grocery Store,Thai Restaurant,Sandwich Place,Ice Cream Shop,Pharmacy,Pizza Place,Café,Furniture / Home Store,Deli / Bodega,...,Music Venue,Donut Shop,Lingerie Store,Supermarket,Liquor Store,Mobile Phone Shop,Mexican Restaurant,Gas Station,Pub,Racetrack
7,Murray Hill,Hotel,Coffee Shop,Japanese Restaurant,Sandwich Place,French Restaurant,Italian Restaurant,Gym,Salon / Barbershop,Cocktail Bar,...,Chinese Restaurant,Burger Joint,Gym / Fitness Center,Bar,Bakery,Bagel Shop,Juice Bar,Mediterranean Restaurant,New American Restaurant,Pub
8,Ocean Parkway,Gym,Pizza Place,General Entertainment,Paper / Office Supplies Store,Steakhouse,Café,Gift Shop,Sushi Restaurant,Bus Station,...,Sake Bar,Grocery Store,Gym / Fitness Center,Restaurant,Bagel Shop,Playground,Fried Chicken Joint,Food,Fast Food Restaurant,Farmers Market


**Now, we create the secondary clustering based on the venues in the neighborhoods. As number of the shortlisted neighborhoods is small and they are similar to each other based on the indicators, we create only two secondary clusters.**

In [411]:
# set number of clusters
kclusters = 2

chosen_neighborhood_clustering = chosen_neighborhood_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chosen_neighborhood_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
chosen_neighborhood_clustering.head()

[1 0 0 1 0 0 0 0 0]


,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bistro,...,Toy / Game Store,Train Station,Used Bookstore,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.090909,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
1,0.058824,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.176471,0.000000,0.058824,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
2,0.012821,0.0,0.012821,0.000000,0.000000,0.012821,0.038462,0.000000,0.102564,0.000000,...,0.000000,0.000000,0.012821,0.012821,0.025641,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.0,0.000000,0.029851,0.029851,0.000000,0.044776,0.000000,0.000000,0.000000,...,0.014925,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.014925


In [412]:
# add clustering labels
chosen_neighborhood_grouped.insert(0, 'Cluster_2', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
chosen_neighborhood_grouped.head(10) # check the last columns!

,Cluster_2,Neighborhood,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,...,Toy / Game Store,Train Station,Used Bookstore,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,1,Bay Terrace,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.090909,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
1,0,Borough Park,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.176471,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
2,0,Bushwick,0.012821,0.000000,0.012821,0.000000,0.000000,0.012821,0.038462,0.000000,...,0.000000,0.000000,0.012821,0.012821,0.025641,0.00,0.00,0.000000,0.000000,0.000000
3,1,East Elmhurst,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
4,0,Flushing,0.000000,0.000000,0.000000,0.029851,0.029851,0.000000,0.044776,0.000000,...,0.014925,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.014925
5,0,Jackson Heights,0.012195,0.012195,0.000000,0.012195,0.000000,0.000000,0.048780,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.012195,0.012195,0.000000
6,0,Kensington,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.028571,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
7,0,Murray Hill,0.020000,0.000000,0.000000,0.000000,0.000000,0.020000,0.020000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.010000,0.01,0.01,0.000000,0.000000,0.000000
8,0,Ocean Parkway,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000


**We see that out of 8 candidate neighborhoods East Elmhurst is the only one classified to the cluster with Bay Terrace. It shall be the best choice for our client.
Now we merge the the datasets containing the coordinates and venues based on the name of the neigborhood. This is necessary to create a map of the shortlisted neighborhoods.**

In [414]:
#chosen_neighborhood_merged=chosen_neighborhood_grouped.merge(Chosen_neighborhoods,on='Neighborhood')
chosen_neighborhood_merged.head()

,Cluster_2,Neighborhood,American Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,...,Street_clean,Rent_Burden,Commute_time,Unemployment,Poverty,Industrial_area,Borough,Latitude,Longitude,Cluster
0,1,Bay Terrace,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,98.2,55.4,41,3.5,25.2,0.041905,Staten Island,40.553988,-74.139166,2
1,0,Borough Park,0.058824,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.176471,...,86.3,60.6,37,3.5,28.3,0.024619,Brooklyn,40.633131,-73.990498,2
2,0,Bushwick,0.012821,0.0,0.012821,0.000000,0.000000,0.012821,0.038462,0.000000,...,92.1,48.6,40,8.8,25.1,0.050248,Brooklyn,40.698116,-73.925258,2
3,1,East Elmhurst,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,94.8,53.7,41,4.3,25.4,0.003563,Queens,40.764073,-73.867041,2
4,0,Flushing,0.000000,0.0,0.000000,0.029851,0.029851,0.000000,0.044776,0.000000,...,98.2,55.4,41,3.5,25.2,0.041905,Queens,40.764454,-73.831773,2


**Now, let's visualize the shorlisted neighborhoods.**

In [418]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chosen_neighborhood_merged['Latitude'], chosen_neighborhood_merged['Longitude'], chosen_neighborhood_merged['Neighborhood'],chosen_neighborhood_merged['Cluster_2']):
    label = folium.Popup(str(poi) + ' Cluster_2 ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters